In [1]:
import numpy as np
from transformers import BertTokenizer, AutoModel
import torch
import datasets

In [5]:
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir='cache/bert-tokenizer', local_files_only=True)

### 报错原因与解决方法

1. **报错原因**：
- `tqdm.notebook` 依赖于 `ipywidgets` 和 `IProgress`，如果缺失或版本不兼容，会导致 AttributeError 或 ImportError。
- 你的环境缺少 `ipywidgets` 或 `IProgress`，或 Jupyter 版本较低。

2. **解决方法**：
- 在终端执行以下命令升级相关依赖：
  ```bash
  pip install --upgrade jupyter ipywidgets
  jupyter nbextension enable --py widgetsnbextension
  ```
- 重启 Jupyter Notebook。

3. **参考链接**：
- [ipywidgets官方安装文档](https://ipywidgets.readthedocs.io/en/stable/user_install.html)